# The S&P 500® Equal Weight Index (EWI)

## Introduction
The S&P 500® Equal Weight Index (EWI) is the equal-weight version of the widely-used S&P 500. The index includes the same constituents as the capitalization weighted S&P 500, but each company in the S&P 500 EWI is allocated a fixed weight. Although both indexes are comprised of the same stocks, the different weighting schemes result in two indexes with different properties and different benefits for investors.

One of the most important characteristics of the S&P 500 is that it is market capitalization-weighted.
Larger companies get a correspondingly larger weight in the index. This project focuses on building an alternative version of the S&P 500 Index and where each company has the same weighting.

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

## Importing Our List of Stocks
Attached below is a link to an Excel File with the most up to date consituents of the S&P 500. 

https://bit.ly/3jPqQuV

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Accessing the API Token
The data provider, IEX Cloud, is a platform that makes financial data and services accessible to everyone. API tokens should be stored in a private file that doesn't get pushed to your local Git respository. We are going to use a sandbox API token. The data that is used is randomly-generated and free. 


In [5]:
from secrets import IEX_CLOUD_API_TOKEN

## Making the First API Call
In order to create the EWI S&P 500, we need the following information: 
   - Market Capitalization for each Stock
   - Price of each Stock

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data) 

{'avgTotalVolume': 71641716, 'calculationPrice': 'close', 'change': 0.66, 'changePercent': 0.0044, 'close': 0, 'closeSource': 'falcfioi', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': 'oseCl', 'highTime': 1700425495399, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 154.97, 'iexCloseTime': 1666225401727, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 159.71, 'iexOpenTime': 1658309404420, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1695930859318, 'latestPrice': 157.6, 'latestSource': 'Close', 'latestTime': 'September 3, 2021', 'latestUpdate': 1687495634181, 'latestVolume': None, 'low': 0, 'lowSource': 'Colse', 'lowTime': 1665897635393, 'marketCap': 2564212292348, 'oddLotDelayedPrice': None,

## Parsing Our API Call

In [7]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap/1000000000000)

2.564212292348


## Adding Stocks Data to Pandas DataFrame

In [8]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [9]:
final_dataframe.append(
    pd.Series(
    [
      symbol,
      price,
      market_cap, 
      'N/A'
    ],
    index = my_columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,157.6,2564212292348,N/A


# Looping Through the Tickers in Our List of Stocks

In [10]:
final_dataframe = pd.DataFrame(columns = my_columns)

for stock in stocks['Symbol']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
    index = my_columns),
    ignore_index = True
)

In [11]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,185.50,54646080488,N/A
1,AAL,19.63,12580074493,N/A
2,AAP,199.65,13008055382,N/A
3,AAPL,160.10,2651824558818,N/A
4,ABBV,116.51,203450340660,N/A
...,...,...,...,...
500,YUM,136.68,39471880999,N/A
501,ZBH,149.19,31449009837,N/A
502,ZBRA,604.00,32589180477,N/A
503,ZION,59.61,9590037879,N/A


## Using Batch API Calls to Improve Performance


We must find a way to split a list of 500+ tickers into a sublists of lists of a 100  

In [12]:
 def make_sublists(the_list, n):
    # Yields successive n-sized chunks from list
    for  i in range(0, len(the_list), n):
        yield the_list[i: i + n]

In [13]:
# Now we need to use this make_sublists() to create a list of lists where every list is no longer than 100
ticker_sublist = list(make_sublists(stocks['Symbol'], 100))

#List of strings, where each string is a comma separated stock of the ticker_sublist
tickers_strings = []
#Make a for loops that loops through every panda series in that list
for i in range(0, len(ticker_sublist)):
    tickers_strings.append(','.join(ticker_sublist[i]))
   # print(tickers_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)
for ticker_string in tickers_strings:
    #Executes a Batch API call
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={ticker_string},fb,tsla&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #And for every stock in that list, appends the information from that stock to our final data frame
    for ticker in ticker_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                ticker, 
                data[ticker]['quote']['latestPrice'],
                data[ticker]['quote']['marketCap'],
                'N/A'
                ],
                index = my_columns),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,184.88,56873391406,N/A
1,AAL,19.96,12772823050,N/A
2,AAP,203.61,12640551516,N/A
3,AAPL,157.50,2616752681325,N/A
4,ABBV,115.07,199350174974,N/A
...,...,...,...,...
500,YUM,135.81,40411013397,N/A
501,ZBH,153.07,31351363252,N/A
502,ZBRA,597.90,32031679131,N/A
503,ZION,59.61,9688470754,N/A


## Calculating the Number of Shares to Buy

In [14]:
portfolio_amount = input("Enter the value of your portfolio:")
try:
    val = int(portfolio_amount)
except ValueError:
    print("That's not a number! \n Try again:") 
    portfolio_amount = input("Enter the value of your portfolio:")


Enter the value of your portfolio:20000000


In [15]:
position_size = float(portfolio_amount) / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,184.88,56873391406,214
1,AAL,19.96,12772823050,1984
2,AAP,203.61,12640551516,194
3,AAPL,157.50,2616752681325,251
4,ABBV,115.07,199350174974,344
...,...,...,...,...
500,YUM,135.81,40411013397,291
501,ZBH,153.07,31351363252,258
502,ZBRA,597.90,32031679131,66
503,ZION,59.61,9688470754,664


## Formatting Our Excel Output
Xlsxwriter is a Python Module that can write to an Excel spreadsheet


### Initializing our XlsxWriter Object

In [16]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, sheet_name = 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our .xlsx File

In [17]:
#Dark Navy Background
background_color = '#0a0a23'
#White Font
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our .xlsx File

In [18]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

0

### Saving Our Excel Output

In [19]:
writer.save()